Init Container
==============

Init containers are special containers that run in front of app containers in a pod.

The example below defines a simple pod with two init containers.

The first is waiting for `myservice` and the second is waiting for `mydb`. Once both init containers complete, the pod runs the app container in its `spec` section.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  name: myapp-pod
  labels:
    app: myapp
spec:
  containers:
  - name: myapp-container
    image: busybox:1.28
    command: ['sh', '-c', 'echo The app is running! && sleep 3600']
  initContainers:
  - name: init-myservice
    image: busybox:1.28
    command: ['sh', '-c', 'until nslookup myservice; do echo waiting for myservice; sleep 2; done;']
  - name: init-mydb
    image: busybox:1.28
    command: ['sh', '-c', 'until nslookup mydb; do echo waiting for mydb; sleep 2; done;']
%EOF%

Status display

In [ ]:
! kubectl get pods/myapp-pod
! kubectl describe pods/myapp-pod

By creating the service `myservices` the first init container is executed

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: myservice
spec:
  ports:
  - protocol: TCP
    port: 80
    targetPort: 9376
%EOF%

In [ ]:
! kubectl get pods/myapp-pod

And when we set up the second service, the second init container and then My-App are started.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: mydb
spec:
  ports:
  - protocol: TCP
    port: 80
    targetPort: 9377
%EOF%

In [ ]:
! kubectl get pods/myapp-pod
! kubectl logs pods/myapp-pod